<a href="https://colab.research.google.com/github/HWaymentSteele/AF_Cluster/blob/main/AFcluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AF-cluster (in Colab form!)

Last updated H Wayment-Steele, Jan 2023

### To cluster (subsample) a big MSA:

1. Upload the big MSA.

2. Run the "**Cluster MSA**" cell, changing `EX` to your desired keyword and the name of your input a3m. MSAs will be written to "subsampled_MSAs". 

3. Uncomment the correct path (containing `/content/subsampled_MSAs` ) in the "**Run Models**" cell, and run it.

4. Output PDBs will be saved in `output`.

### If you have an existing set of subsampled MSAs to run:
1. Upload a directory of MSA files to run.

2. Change the `AFcluster/data..\*a3m` path in the "**Run Models**" cell and run it.

3. Output PDBs will be saved in `output`.

In [10]:
#@markdown ## Setup dependencies
%%bash

if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi

if [ ! -d AFcluster ];then
  git clone https://github.com/HWaymentSteele/AFcluster.git
fi

if [ ! -d alphafold ]; then
  git clone https://github.com/deepmind/alphafold.git
  ! pip -q install ml-collections dm-haiku biopython
fi

if [ ! -d output ]; then
  mkdir output
fi

Cloning into 'AFcluster'...
Checking out files: 100% (13504/13504), done.


In [1]:
#@markdown ## Define functions to run AlphaFold

import sys
import os
import argparse
import hashlib
import jax
import jax.numpy as jnp
import numpy as np
import re
import subprocess
from glob import glob

sys.path.append('alphafold')

from alphafold.model import model, config, data
from alphafold.data import parsers, pipeline
from alphafold.common import protein

"""
Create an AlphaFold model runner
name -- The name of the model to get the parameters from. Options: model_[1-5]
"""

def make_model_runner(model_num=3, recycles=1, deterministic=True):
  model_name = 'model_%d_ptm' % model_num
  cfg = config.model_config(model_name)      

  cfg.data.common.num_recycle = recycles
  cfg.model.num_recycle = recycles
  cfg.data.eval.num_ensemble = 1
  if deterministic:
    cfg.data.eval.masked_msa_replace_fraction = 0.0
    cfg.model.global_config.deterministic = True
  params = data.get_model_haiku_params(model_name, '.')

  return model.RunModel(cfg, params)

def make_processed_feature_dict(runner, a3m_file, name="test", seed=0):
  feature_dict = {}

  # assumes sequence is first entry in msa
  with open(a3m_file,'r') as msa_fil:
    sequence = msa_fil.read().splitlines()[1].strip()

  feature_dict.update(pipeline.make_sequence_features(sequence, name, len(sequence)))

  with open(a3m_file,'r') as msa_fil:
    msa = pipeline.parsers.parse_a3m(msa_fil.read())

  feature_dict.update(pipeline.make_msa_features([msa]))
  processed_feature_dict = runner.process_features(feature_dict, random_seed=seed)

  return processed_feature_dict

"""
Package AlphaFold's output into an easy-to-use dictionary
prediction_result - output from running AlphaFold on an input dictionary
processed_feature_dict -- The dictionary passed to AlphaFold as input. Returned by `make_processed_feature_dict`.
"""
def parse_results(prediction_result, processed_feature_dict):
  b_factors = prediction_result['plddt'][:,None] * prediction_result['structure_module']['final_atom_mask']  
  dist_bins = jax.numpy.append(0,prediction_result["distogram"]["bin_edges"])
  dist_mtx = dist_bins[prediction_result["distogram"]["logits"].argmax(-1)]
  contact_mtx = jax.nn.softmax(prediction_result["distogram"]["logits"])[:,:,dist_bins < 8].sum(-1)

  out = {"unrelaxed_protein": protein.from_prediction(processed_feature_dict, prediction_result, b_factors=b_factors),
        "plddt": prediction_result['plddt'],
        "pLDDT": prediction_result['plddt'].mean(),
        "dists": dist_mtx,
        "adj": contact_mtx}

  out.update({"pae": prediction_result['predicted_aligned_error'],
              "pTMscore": prediction_result['ptm']})
  return out

def write_results(result, pdb_out_path):
  plddt = float(result['pLDDT'])
  ptm = float(result["pTMscore"])
  print('plddt: %.3f' % plddt)
  print('ptm  : %.3f' % ptm)

  pdb_lines = protein.to_pdb(result["unrelaxed_protein"])
  with open(pdb_out_path, 'w') as f:
    f.write(pdb_lines)

/usr/local/lib/python3.8/dist-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


# **Cluster MSA**

In [6]:
%%bash

pip -q install polyleven

python AFcluster/scripts/ClusterMSA.py EX -i AFcluster/data_sep2022/00_KaiB/2QKEE_colabfold.a3m -o subsampled_MSAs

EX
1021 seqs removed for containing more than 25% gaps, 7137 remaining
eps	n_clusters	n_not_clustered
3.00	4	1755
3.50	6	1743
4.00	7	1733
4.50	13	1687
5.00	18	1656
5.50	38	1554
6.00	41	1479
6.50	57	1268
7.00	70	1066
7.50	79	771
8.00	65	560
8.50	58	324
9.00	25	174
9.50	7	63
10.00	2	20
10.50	1	0
Selected eps=7.50
7137 total seqs
245 clusters, 1754 of 7137 not clustered (0.25)
avg identity to query of unclustered: 0.38
avg identity to query of clustered: 0.43
writing 10 size-10 uniformly sampled clusters
writing 10 size-100 uniformly sampled clusters
wrote clustering data to subsampled_MSAs/EX_clustering_assignments.tsv
wrote cluster metadata to subsampled_MSAs/EX_cluster_metadata.tsv
Saved this output to EX.log


In [2]:
n_recycles = 3
model_number = 3
seed=0
name='KaiB_TE'

runner = make_model_runner(model_num=model_number, recycles=n_recycles)

subsampled_msas = glob('AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/*a3m')
#subsampled_msas = glob('subsampled_MSAs/*a3m')

for fil in subsampled_msas:
  print(fil)
  features = make_processed_feature_dict(runner, fil, seed=seed)
  result = parse_results(runner.predict(features, random_seed=seed), features)
  write_results(result, 'output/' + os.path.basename(fil).replace('.a3m','.pdb'))

AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U10-103.a3m
plddt: 68.423
ptm  : 0.492
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U100-476.a3m
plddt: 87.045
ptm  : 0.763
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U100-120.a3m
plddt: 84.589
ptm  : 0.740
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U10-488.a3m
plddt: 85.976
ptm  : 0.752
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_06.a3m
plddt: 66.378
ptm  : 0.486
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U100-190.a3m
plddt: 88.812
ptm  : 0.802
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_130.a3m
plddt: 65.222
ptm  : 0.498
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U10-462.a3m
plddt: 81.327
ptm  : 0.684
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U100-069.a3m
plddt: 90.948
ptm  : 0.833
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U10-035.a3m
plddt: 88.240
ptm  : 0.795
AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/2QKEE_U10-456.a3m
plddt: 86

KeyboardInterrupt: ignored